In [ ]:
#Input parameters
weatherStationSerial = "BWEATHER-3"
citySerial = "Loudeac"
pPlantSerial = "JPP-L-2"
batterySerial = "JPP-S-BS"
energyTraderSerial = "ET-1"
quantileValue = "0.1"
#Example parameter string for default parameters {"weatherStationSerial":"BWEATHER-1","citySerial":"Loudeac","pPlantSerial":"JPP-1","batterySerial":"BAT-1"}
#                                                {"weatherStationSerial":"BWEATHER-1","citySerial":"Loudeac","pPlantSerial":"JPP-S-1","batterySerial":"BAT-S-1"}



In [ ]:
#Dynamically fetch the correct battery
quantileValue = float(quantileValue)
pPlantToBatSerialDict = {"1":'BS',
                         "2":'BM',
                         "3":'BL'}

pPlantSerialArray = pPlantSerial.split("-")
pPlantSerialArray[2] = pPlantToBatSerialDict[pPlantSerialArray[2]]


In [ ]:
batterySerial = "-".join(pPlantSerialArray)

In [ ]:
#!pip install --upgrade git+https://alroc@bitbucket.org/freemens/ionlib.git@master

In [ ]:
#!pip install -qqq --upgrade git+https://bitbucket.org/freemens/ion_sdk.git@master

In [ ]:
#!pip install -qqq --upgrade pandas
#!pip install -qqq --upgrade numpy

In [ ]:
%env ALTERGO_FACTORY_API = https://staging.altergo.io/
%env ALTERGO_IOT_API = https://iot.staging.altergo.io/   

In [ ]:
from ionlib.models.battery import Battery, Cell, LEVEL_MODULE
from random import random
from datetime import datetime
from ionlib.tools.toolbox import interpolateValue

In [ ]:
from ionlib.models.battery import Battery, Cell, LEVEL_MODULE
from ionlib.models.powerplant import PowerPlant,SolarArray
import pandas as pd
import ion_sdk.edison_api.edison_api as eapi
from ion_sdk.edison_api.models.factoryModel import EdisonGenericComponent, Model,CurrentState
from ion_sdk.tools.toolbox import plotSensors, dataframeFromSensors
from ion_sdk.tools.sim import Sensor
import datetime
from datetime import datetime
import math
from ionlib.specs.powerplant_specs import simSpecs
import pprint


In [ ]:
edApi = eapi.Client("f6ad4f7166814976b66b9cb346ebc5dd")

In [ ]:
powerPlantAsset = edApi.getAsset(pPlantSerial)
solarPanelSurface = powerPlantAsset.model.parameters[0].parameter_values[0].value


In [ ]:
batteryAsset = edApi.getAsset(batterySerial)

batteryVoltage = 400
batteryCapacity = 50
neededParameters = ["Voltage","Capacity","Current (charge)","Current (discharge)"]
parameterDict = {}
for param in batteryAsset.model.parameters:
    if param.parameter_model.name in neededParameters:
        parameterDict[param.parameter_model.name] = param.parameter_values[0].value
        


In [ ]:
moduleInSeries = float(parameterDict["Voltage"])/float(simSpecs['cell']['voltage']) / float(simSpecs['architecture']['cellsSeries'])
moduleInSeries = math.ceil(moduleInSeries)

moduleInPar=float(parameterDict["Capacity"])/float(simSpecs['cell']['cellCapacity'])/14
moduleInPar = math.ceil(moduleInPar)

simSpecs['architecture']["modulesSeries"] = moduleInSeries
simSpecs['architecture']["modulesParallel"] = moduleInPar

In [ ]:
weatherStationAsset = edApi.getAsset(weatherStationSerial)
sensorNameList = edApi.matchSensors(["Direct radiation","Radiation rolling average"],weatherStationAsset)
req={
    "assets":[weatherStationAsset],
    "sensorNames":sensorNameList,
    "startDate":eapi.edisonDate(2020,1,1,0,00),
    "endDate":eapi.edisonDate(2020,12,31,0,00)
        }

edApi.getAssetDataFrame(**req)

weatherStationDf = weatherStationAsset.df

In [ ]:
from datetime import timedelta

In [ ]:
quantile = weatherStationDf["Radiation rolling average"].quantile(quantileValue)
result_index = weatherStationDf['Radiation rolling average'].sub(quantile).abs().idxmin()
pollingStartDate = result_index.timestamp()* 1000
pollingEndDate = result_index + timedelta(days=7)
pollingEndDate = pollingEndDate.timestamp() * 1000


In [ ]:
traderAsset = edApi.getAsset(energyTraderSerial)
sensorNameList = edApi.matchSensors(["Price"],traderAsset)
req={
    "assets":[traderAsset],
    "sensorNames":sensorNameList,
    "startDate":pollingStartDate,
    "endDate":pollingEndDate
        }

edApi.getAssetDataFrame(**req)

traderAssetDf = traderAsset.df

In [ ]:
cityAsset = edApi.getAsset(citySerial)
sensorNameList = edApi.matchSensors(["Power"],cityAsset)
req={
    "assets":[cityAsset],
    "sensorNames":sensorNameList,
    "startDate":pollingStartDate,
    "endDate":pollingEndDate
        }

edApi.getAssetDataFrame(**req)

cityDf = cityAsset.df

In [ ]:
SIM_TARGETED_DEPTH = LEVEL_MODULE
simTime = pollingEndDate / 1000
t = pollingStartDate / 1000
dt_s = 60.0
prev_dt_s = 60.0
time = []

pPlant = PowerPlant(sourceProfile = weatherStationDf, loadProfile = cityDf, simSpecs = simSpecs, prices=traderAssetDf)

pPlant.solarArray.surface = float(solarPanelSurface)
pPlant.maxAcp = float(parameterDict['Current (charge)'])
pPlant.maxAdp = float(parameterDict['Current (discharge)'])

In [ ]:
analysedElements = []
pPlant.battery.getElementsByDepth(analysedElements, SIM_TARGETED_DEPTH)

In [ ]:
pPlantSensors = {}
batterySensors = {}
loadSensors = {}
tradingSensors = {}

pPlantSensors["sales"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "°",
        "axis": 1,
        "synced": 1,
        "plotted": 1,
    }
)

pPlantSensors["production potential"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "W",
        "axis": 1,
        "synced": 1,
        "plotted": 1,
    }
)
pPlantSensors["demand potential"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "W",
        "axis": 1,
        "synced": 1,
        "plotted": 1,
    }
)
pPlantSensors["production actual"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "W",
        "axis": 1,
        "synced": 1,
        "plotted": 1,
    }
)
pPlantSensors["power"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "°",
        "axis": 1,
        "synced": 1,
        "plotted": 1,
    }
)
pPlantSensors["over production"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "W",
        "axis": 1,
        "synced": 1,
        "plotted": 1,
    }
)
pPlantSensors["blackout"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "W",
        "axis": 2,
        "synced": 1,
        "plotted": 1,
    }
)




In [ ]:
batterySensors["current"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "A",
        "axis": 2,
        "synced": 1,
        "plotted": 1,
    }
)
batterySensors["power"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "W",
        "axis": 3,
        "synced": 1,
        "plotted": 1,
    }
)
batterySensors["soc"] = Sensor(
    **{
        "data": [],
        "time": [],
        "lastSigVal": 0.0,
        "sigVarTh": 0.001,
        "unit": "%",
        "axis": 1,
        "synced": 1,
        "plotted": 1,
    }
)

In [ ]:

while t < simTime:
# Manage battery current
    current = pPlant.managePower(t)

    # Simulation time step management
    dt_s = min(
        60.0,
        float(
            pPlant.battery.capacity.init * 3600.0 / (1.0 + abs(current * 400.0))
        ),
    )
    if dt_s > prev_dt_s:
        dt_s = prev_dt_s * 1.2
        if dt_s >= 60.0:
            dt_s = 60.0

    prev_dt_s = dt_s
    t = t + dt_s

    pPlant.battery.setCurrent(current, SIM_TARGETED_DEPTH)

    if t == simSpecs["start"]:
        pPlant.salesMin = 0 - (80*self.solarArray.surface)
    else:
        pPlant.calculatePricing(t,dt_s)
    
    for l in analysedElements:
        l.calculateNextStep(dt_s, 25.0)
    pPlant.battery.soc = min(l.soc for l in analysedElements)

    tDate = datetime.fromtimestamp(t)



    batterySensors["soc"].significantAppend(float(pPlant.battery.soc*100), tDate)
    batterySensors["power"].significantAppend(float(pPlant.pBat), tDate)
    batterySensors["current"].significantAppend(float(current), tDate)

    pPlantSensors['sales'].significantAppend(float(pPlant.salesMin), tDate)
    pPlantSensors["production potential"].significantAppend(float(pPlant.pProductionPot), tDate)
    pPlantSensors["demand potential"].significantAppend(float(pPlant.pDemandPot), tDate)
    pPlantSensors["production actual"].significantAppend(float(pPlant.pProduction), tDate)
    pPlantSensors["power"].significantAppend(float(pPlant.pSupplied), tDate)
    pPlantSensors["over production"].significantAppend(float(pPlant.overProduction), tDate)
    pPlantSensors["blackout"].significantAppend(float(pPlant.isBlackout), tDate)


In [ ]:
batteryDf = dataframeFromSensors(batterySensors)
pPlantDf = dataframeFromSensors(pPlantSensors)
pPlantDf['Balance'] = pPlantDf['sales'].cumsum()
pPlantDf['Cumulated blackout time'] = pPlantDf['blackout'].cumsum()

In [ ]:
try:
    etAsset = edApi.getAsset(energyTraderSerial)
    etAsset.df = salesDf
    uploadSensorList = edApi.refactorDataframeToAsset(etAsset.df, etAsset)
    edApi.updateSensorDataByFile(etAsset, uploadSensorList)
except Exception as e:
    print(etAsset)
    print(e)

In [ ]:
try:
    batAsset = edApi.getAsset(batterySerial)
    batAsset.df = batteryDf
    uploadSensorList = edApi.refactorDataframeToAsset(batAsset.df, batAsset)
    edApi.updateSensorDataByFile(batAsset, uploadSensorList)
except Exception as e:
    print(batteryDf)
    print(e)

In [ ]:
try:
    pPlantAsset = edApi.getAsset(pPlantSerial)
    pPlantAsset.df = pPlantDf
    uploadSensorList = edApi.refactorDataframeToAsset(pPlantAsset.df, pPlantAsset)
    edApi.updateSensorDataByFile(pPlantAsset, uploadSensorList)
except Exception as e:
    print(pPlantDf)
    print(e)

In [ ]:
#df = dataframeFromSensors(batterySensors)
#fig = plotSensors(batterySensors, df, "Test")
#fig.show()

In [ ]:
#df = dataframeFromSensors(pPlantSensors)
#fig = plotSensors(pPlantSensors, df, "Test")
#fig.show()